In [23]:
# absolute imports
import json, os, sys
from pathlib import Path

# relative imports
sys.path.append(os.getcwd() + "/..")
from python.lumifit.general import load_params_from_file
from python.lumifit.experiment import Experiment
from python.lumifit.simulation import generateDirectory, SimulationParameters
from python.lumifit.reconstruction import ReconstructionParameters, generateRecoDirSuffix
from python.lumifit.alignment import AlignmentParameters

In [24]:
def getLumiDevFromExp(exp: Experiment) -> float:

    alignParams = exp.alignParams
    recoParams = exp.recoParams
    simParams = exp.simParams

    # recoDir = Path(generateDirectory(simParams, alignParams)) / generateRecoDirSuffix(recoParams, alignParams)
    recoDir = exp.baseDataOutputDir / generateRecoDirSuffix(recoParams, alignParams)
    lumiValuesFile = Path(recoDir).glob("bunches_*/binning_*/merge_data/lumi*")

    for res in lumiValuesFile:
        with open(res, "r") as f:
            lumiValues = json.load(f)
        return round(float(lumiValues["relative_deviation_in_percent"]), 2)

    return None

In [25]:
def getRecoIP(exp: Experiment):
    recoDir = exp.baseDataOutputDir / "1-100_uncut/no_alignment_correction"
    lumiValuesFile = Path(recoDir).glob("bunches_*/binning_*/merge_data/reco_ip*")

    for res in lumiValuesFile:
        with open(res, "r") as f:
            lumiValues = json.load(f)
        return round(float(lumiValues["ip_x"])*1e4, 2), round(float(lumiValues["ip_y"])*1e4, 2)
        


In [26]:
configPath = Path("../python/expConfigs/PANDA/restrictPhi/")

print('| Mom   | Matrix                | Deviation | IP Pos. |\n| ----- | --------------------- | --------- | - |')

for file in configPath.iterdir():
    exp = load_params_from_file(str(file), Experiment)
    exp.recoParams.use_ip_determination = True

    # for now, those sims don't exist
    if exp.simParams.lab_momentum == 11.91 or exp.simParams.lab_momentum == 4.06:
        continue

    print(f'| {exp.recoParams.lab_momentum} | {exp.alignParams.misalignment_matrices_path.stem} | {getLumiDevFromExp(exp)} | {getRecoIP(exp)} | ')
    # break


| Mom   | Matrix                | Deviation | IP Pos. |
| ----- | --------------------- | --------- | - |
| 15.0 | misMat-nothing-2piO2 | 0.23 | (-98.12, 35.19) | 
| 8.9 | misMat-nothing-2piO8 | -2.49 | (-25.32, 32.86) | 
| 15.0 | misMat-nothing-2piO10 | -13.34 | (-51.5, 16.6) | 
| 1.5 | misMat-nothing-2piO12 | -2.7 | (-110.42, 22.46) | 
| 8.9 | misMat-nothing-2pi | -0.12 | (-69.05, 1.84) | 
| 8.9 | misMat-nothing-2piO4 | -0.61 | (-44.74, 37.49) | 
| 1.5 | misMat-nothing-2pi | -1.24 | (-107.65, 32.61) | 
| 1.5 | misMat-nothing-2piO4 | -0.12 | (-108.16, 11.06) | 
| 15.0 | misMat-nothing-2piO4 | -0.27 | (-69.41, 28.64) | 
| 1.5 | misMat-nothing-2piO8 | -6.36 | (-93.67, 24.77) | 
| 15.0 | misMat-nothing-2piO8 | -1.45 | (-56.86, 22.04) | 
| 1.5 | misMat-nothing-2piO10 | -12.91 | (-112.24, 16.8) | 
| 8.9 | misMat-nothing-2piO12 | -7.22 | (-27.28, 21.21) | 
| 8.9 | misMat-nothing-2piO10 | -9.2 | (-26.61, 26.49) | 
| 15.0 | misMat-nothing-2pi | -1.28 | (-94.03, 4.12) | 
| 8.9 | misMat-nothing